In [96]:
# Import libraries
import scipy.io
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import normalize


# Define the multilayer perceptron (MLP) class
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        # Create a list of linear layers
        self.layers = nn.ModuleList()
        # Add the first layer (from input to first hidden layer)
        self.layers.append(nn.Linear(input_size, hidden_sizes[0]))
        # Add the remaining hidden layers
        for i in range(len(hidden_sizes) - 1):
            self.layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
        # Add the output layer
        self.layers.append(nn.Linear(hidden_sizes[-1], output_size))

    def forward(self, x):
        # Loop through the hidden layers
        for layer in self.layers[:-1]:
            # Apply linear transformation and ReLU activation
            x = F.sigmoid(layer(x))
            x = nn.Dropout(p=0.2)(x)
        # Apply the output layer and sigmoid activation
        x = torch.squeeze(torch.sigmoid(self.layers[-1](x)))
        return x

# Define the radial basis function (RBF) class
class RBF(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, radius):
        super(RBF, self).__init__()
        # Initialize the centers and sigmas of the RBFs randomly
        self.centers = nn.Parameter(torch.randn(hidden_size, input_size))
        self.sigmas = nn.Parameter(torch.randn(hidden_size)*radius)
        # Initialize the output layer
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Compute the pairwise distances between x and centers
        dists = torch.cdist(x, self.centers)
        # Compute the RBF outputs
        rbfs = torch.exp(-self.sigmas * dists)
        # Apply the output layer and sigmoid activation
        x = torch.squeeze(torch.sigmoid(self.linear(rbfs)))
        return x

# Load the data (you need to provide your own data here)

# Convert the data to tensors

X = scipy.io.loadmat('X.mat')['X']
X = np.squeeze(X);
Xtst = scipy.io.loadmat('Xtst.mat')['Xtst']
Xtst = np.squeeze(Xtst);
y = scipy.io.loadmat('TrainLabels.mat')['TrainLabels'].T
y = np.squeeze(y);
y[0] = 1
y = np.round(y/2+0.5)
Indexes = scipy.io.loadmat('Index.mat')['Index'].T
Indexes = np.squeeze(Indexes);
X = normalize(X)
print(X.shape)
n_select = 200
X = X[:, Indexes[:n_select]]
Xtst = Xtst[:, Indexes[:n_select]]

X = torch.from_numpy(X).float()
Xtst = torch.from_numpy(Xtst).float()
y = torch.from_numpy(y).float()

print(X.shape)

# Define the number of folds for cross validation
n_folds = 5

# Define the range of number of neurons and layers to iterate over
n_neurons = [3, 4,5,6,7, 10, 20, 30, 40]
radius = [ 0.1, 0.1, 0.1, 0.1]
#n_neurons = [5, 10, 20, 30, 40]
n_layers = [2,3,4, 5]

# Initialize the best accuracy and the best parameters
best_acc_mlp = 0
best_params_mlp = None
best_acc_rbf = 0
best_params_rbf = None
model_best = None

# Loop over the number of neurons
ind = -1
for l in n_layers:
    # Loop over the number of layers
    ind += 1
    for n in n_neurons:
        # Create a list of hidden sizes for the MLP
        hidden_sizes = [n] * l
        # Initialize the MLP and RBF models

        #rbf = RBF(input_size=X.shape[1], hidden_size=n, output_size=1, radius=radius[ind])
        # Initialize the cross validation scores
        mlp_scores = []
        rbf_scores = []
        # Create a KFold object
        kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
        # Loop over the folds
        for train_index, test_index in kf.split(X):
            mlp = MLP(input_size=X.shape[1], hidden_sizes=hidden_sizes, output_size=1)
            # Split the data into train and test sets
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            # Define the loss function and the optimizer
            criterion = nn.BCELoss()
            mlp_optimizer = optim.Adam(mlp.parameters())
            #rbf_optimizer = optim.Adam(rbf.parameters())
            # Train the MLP model
            mlp.train()
            for epoch in range(100):
                # Zero the parameter gradients
                mlp_optimizer.zero_grad()
                # Forward pass
                mlp_output = mlp(X_train)
                # Compute the loss
                mlp_loss = criterion(mlp_output, y_train)
                # Backward pass and optimize
                mlp_loss.backward()
                mlp_optimizer.step()
            # Train the RBF model
            #rbf.train()
            #for epoch in range(100):
            #    # Zero the parameter gradients
            #    rbf_optimizer.zero_grad()
                # Forward pass
            #    rbf_output = rbf(X_train)
                # Compute the loss
            #    rbf_loss = criterion(rbf_output, y_train)
                # Backward pass and optimize
            #    rbf_loss.backward()
            #    rbf_optimizer.step()
            # Evaluate the MLP model
            mlp.eval()
            with torch.no_grad():
                # Predict the test set
                mlp_pred = mlp(X_test)
                # Convert the predictions to binary labels
                mlp_pred = (mlp_pred > 0.5).float()
                # Compute the accuracy score
                mlp_score = accuracy_score(y_test, mlp_pred)
                # Append the score to the list
                mlp_scores.append(mlp_score)
            # Evaluate the RBF model
         #   rbf.eval()
         #   with torch.no_grad():
                # Predict the test set
          #      rbf_pred = rbf(X_test)
                # Convert the predictions to binary labels
           #     rbf_pred = (rbf_pred > 0.5).float()
                # Compute the accuracy score
            #    rbf_score = accuracy_score(y_test, rbf_pred)
                # Append the score to the list
            #    rbf_scores.append(rbf_score)
        # Compute the average scores across the folds
        mlp_mean = np.mean(mlp_scores)
       # rbf_mean = np.mean(rbf_scores)
        # Print the scores
        print(f"MLP with {l} layers and {n} neurons: {mlp_mean:.4f}")
       # print(f"RBF with r={radius[ind]} and {n} neurons: {rbf_mean:.4f}")
        # Check if the MLP score is better than the best accuracy
        if mlp_mean > best_acc_mlp:
            # Update the best accuracy and the best parameters
            best_acc_mlp = mlp_mean
            best_params_mlp = ("MLP", l, n)
            model_best = mlp
        # Check if the RBF score is better than the best accuracy
       # if rbf_mean > best_acc_rbf:
            # Update the best accuracy and the best parameters
         #   best_acc_rbf = rbf_mean
         #   best_params_rbf = ("RBF", n)
# Print the best accuracy and the best parameters
print(f"Best accuracy: {best_acc_mlp:.4f}")
print(f"Best parameters: {best_params_mlp}")
##print(f"Best accuracy: {best_acc_rbf:.4f}")
print(f"Best parameters: {best_params_rbf}")

model_best.eval()
with torch.no_grad():
    # Predict the test set
    mlp_pred = model_best(Xtst)
    # Convert the predictions to binary labels
    mlp_pred = (mlp_pred > 0.5).float()



(550, 24145)
torch.Size([550, 200])
MLP with 2 layers and 3 neurons: 0.4909
MLP with 2 layers and 4 neurons: 0.4945
MLP with 2 layers and 5 neurons: 0.5164
MLP with 2 layers and 6 neurons: 0.4982
MLP with 2 layers and 7 neurons: 0.4927
MLP with 2 layers and 10 neurons: 0.5018
MLP with 2 layers and 20 neurons: 0.5036
MLP with 2 layers and 30 neurons: 0.4782
MLP with 2 layers and 40 neurons: 0.5018
MLP with 3 layers and 3 neurons: 0.5036
MLP with 3 layers and 4 neurons: 0.5164
MLP with 3 layers and 5 neurons: 0.5091
MLP with 3 layers and 6 neurons: 0.4727
MLP with 3 layers and 7 neurons: 0.4855
MLP with 3 layers and 10 neurons: 0.4782
MLP with 3 layers and 20 neurons: 0.4836
MLP with 3 layers and 30 neurons: 0.4473
MLP with 3 layers and 40 neurons: 0.4909
MLP with 4 layers and 3 neurons: 0.5236
MLP with 4 layers and 4 neurons: 0.5018
MLP with 4 layers and 5 neurons: 0.4836
MLP with 4 layers and 6 neurons: 0.5018
MLP with 4 layers and 7 neurons: 0.5164
MLP with 4 layers and 10 neurons: 0.

In [100]:
# Import libraries
import torch
import pandas as pd

# Save the tensor as a binary file using torch.save
torch.save(mlp_pred, "tensor.pt")

# Load the tensor from the file using torch.load
t = torch.load("tensor.pt")

# Convert the tensor to a numpy array using .numpy()
t_np = t.numpy()

# Convert the numpy array to a dataframe using pd.DataFrame()
df = pd.DataFrame(t_np)

# Save the dataframe as a csv file using .to_csv()
df.to_csv("TestLabel_mlp.csv", index=False)

# Load the dataframe from the csv file using pd.read_csv()
df = pd.read_csv("TestLabel_mlp.csv")

# New Section